In [1]:
import pandas as pd
import icd10
import numpy as np

import warnings
from causalnex.structure import StructureModel
warnings.filterwarnings("ignore")  # silence warnings

from IPython.display import Image
from causalnex.plots import plot_structure, NODE_STYLE, EDGE_STYLE

from mlxtend.preprocessing import TransactionEncoder

import numpy as np
import pandas as pd
from scipy.stats import norm
from zepid import RiskRatio


# Data Processing

In [2]:
data = pd.read_table('entire_data_participant.tsv')
data = data.rename(columns={"p41270": "Diagnoses - ICD10", "p31": "Sex", 'p34': 'Year of birth'})

In [3]:
data['icd'] = data ['Diagnoses - ICD10'].str.split('|')
data['age'] = 2023 - data['Year of birth']

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
data['gender'] = le.fit_transform(data['Sex'])

In [4]:
data_icd = data[data['icd'].notnull()]

In [5]:
data_no = data[data['icd'].isnull()]

In [6]:
fp_pn =  list(data_icd['icd'])

te = TransactionEncoder()
te_ary = te.fit(fp_pn).transform(fp_pn)
df = pd.DataFrame(te_ary, columns=te.columns_)
df = df.astype(int)

In [7]:
d = pd.DataFrame(0, index=np.arange(len(data_no)), columns=df.columns)
df = pd.concat([df,d])

In [8]:
n = df.describe().T
#top = n.sort_values(by = 'mean', ascending = False)[:500]
top = n[n['mean'] >= 0.005]
test_top = df[top.index]

In [10]:
test_top = df[top.index]

In [11]:
test_top = test_top.reset_index()

In [12]:
data = pd.concat([data_icd,data_no]).reset_index()

In [13]:
df = pd.concat([test_top.iloc[:,1:], data[['age','gender']]], axis=1)

In [14]:
df['age'] = np.where(df['age'] > 65, 1, 0)

In [15]:
df = df.dropna(how='any',axis=0) 

In [110]:
#gold = ['F03','I10','E78.0','N17.9','J22','F41.9','I25.9','N18.3','E66.9','K59','J22','N18.9','N18.3',\
#        'I67.8','G20','I67.9']

# NOTEARS_alpha

In [ ]:
import numpy as np
import scipy.linalg as slin
import scipy.optimize as sopt
from scipy.special import expit as sigmoid

import time


def notears_linear(X, lambda1, loss_type, max_iter=100, h_tol=1e-8, rho_max=1e+16, w_threshold=0.0):
    """Solve min_W L(W; X) + lambda1 ‖W‖_1 s.t. h(W) = 0 using augmented Lagrangian.

    Args:
        X (np.ndarray): [n, d] sample matrix
        lambda1 (float): l1 penalty parameter
        loss_type (str): l2, logistic, poisson
        max_iter (int): max num of dual ascent steps
        h_tol (float): exit if |h(w_est)| <= htol
        rho_max (float): exit if rho >= rho_max
        w_threshold (float): drop edge if |weight| < threshold

    Returns:
        W_est (np.ndarray): [d, d] estimated DAG
    """
    def _loss(W):
        """Evaluate value and gradient of loss."""
        M = X @ W
        if loss_type == 'l2':
            alpha = 0
            Y = 1 - X
            R = X *(X - M) + alpha * Y * (X - M)
            #R = X - M
            loss = 0.5 / X.shape[0] * (R ** 2).sum()
            G_loss = - 1.0 / X.shape[0] * X.T @ R
        elif loss_type == 'logistic':
            loss = 1.0 / X.shape[0] * (np.logaddexp(0, M) - X * M).sum()
            G_loss = 1.0 / X.shape[0] * X.T @ (sigmoid(M) - X)
        elif loss_type == 'poisson':
            S = np.exp(M)
            loss = 1.0 / X.shape[0] * (S - X * M).sum()
            G_loss = 1.0 / X.shape[0] * X.T @ (S - X)
        else:
            raise ValueError('unknown loss type')
        return loss, G_loss

    def _h(W):
        """Evaluate value and gradient of acyclicity constraint."""
        E = slin.expm(W * W)  # (Zheng et al. 2018)
        h = np.trace(E) - d
        #     # A different formulation, slightly faster at the cost of numerical stability
        #     M = np.eye(d) + W * W / d  # (Yu et al. 2019)
        #     E = np.linalg.matrix_power(M, d - 1)
        #     h = (E.T * M).sum() - d
        G_h = E.T * W * 2
        return h, G_h

    def _adj(w):
        """Convert doubled variables ([2 d^2] array) back to original variables ([d, d] matrix)."""
        return (w[:d * d] - w[d * d:]).reshape([d, d])

    def _func(w):
        """Evaluate value and gradient of augmented Lagrangian for doubled variables ([2 d^2] array)."""
        W = _adj(w)
        loss, G_loss = _loss(W)
        h, G_h = _h(W)
        obj = loss + 0.5 * rho * h * h + alpha * h + lambda1 * w.sum()
        G_smooth = G_loss + (rho * h + alpha) * G_h
        g_obj = np.concatenate((G_smooth + lambda1, - G_smooth + lambda1), axis=None)
        return obj, g_obj

    n, d = X.shape
    w_est, rho, alpha, h = np.zeros(2 * d * d), 1.0, 0.0, np.inf  # double w_est into (w_pos, w_neg)
    bnds = [(0, 0) if i == j else (0, None) for _ in range(2) for i in range(d) for j in range(d)]
    if loss_type == 'l2':
        X = X - np.mean(X, axis=0, keepdims=True)
    for iter in range(max_iter):
        print(iter)
        w_new, h_new = None, None
        while rho < rho_max:
            sol = sopt.minimize(_func, w_est, method='L-BFGS-B', jac=True, bounds=bnds,\
                                options={'disp': True,'eps':1e-40,'maxls':40})
            #sol = sopt.minimize(_func, w_est, method='BFGS', jac=True, bounds=bnds)
            #sol = sopt.fmin_l_bfgs_b(_func, w_est,approx_grad=True, bounds=[(1e-8, 0.5)], factr=1e02, pgtol=1e-05, epsilon=1e-08)

            w_new = sol.x
            h_new, _ = _h(_adj(w_new))
            if h_new > 0.25 * h:
                rho *= 10
            else:
                break
        w_est, h = w_new, h_new
        W_est = _adj(w_est)
        print(np.sum(W_est))
        alpha += rho * h
        if h <= h_tol or rho >= rho_max:
            break
    W_est = _adj(w_est)
    return W_est

def read_file(data_dir, sep=","):
    with open(data_dir, "r") as f:
        data = f.read().strip().split("\n")
    data = [i.split(sep) for i in data]
    return np.array(data).astype(float)

if __name__ == '__main__':
    # from tools import utils
    #
    # utils.set_random_seed(1)
    #
    # n, d, s0, graph_type, sem_type = 20000, 100, 20, 'SF', 'gauss' # sem_type: gauss, logistic
    # B_true = utils.simulate_dag(d, s0, graph_type)
    # print(B_true.sum())
    # W_true = B_true
    # #W_true = utils.simulate_parameter(B_true) # why should we simulate parameters?
    # #np.savetxt('W_true.csv', W_true, delimiter=',')
    #
    # X = utils.simulate_linear_sem(W_true, n, sem_type)
    #
    # X2 = utils.simulate_linear_sem(W_true, n, sem_type)
    # #X = utils.simulate_nonlinear_sem(W_true, n, sem_type)
    # #np.savetxt('X.csv', X, delimiter=',')
    #
    # start_time = time.time()
    #
    # W_est = notears_linear(X, lambda1=0., loss_type='l2', max_iter=10)
    #
    # end_time = time.time()
    # print("time: {}".format(end_time-start_time))
    #
    # print("W_est.sum: {}".format(W_est.sum()))
    # W_est[np.abs(W_est) < 0.1] = 0
    # assert utils.is_dag(W_est)
    # #np.savetxt('W_est.csv', W_est, delimiter=',')
    # acc = utils.count_accuracy(B_true, W_est != 0)
    # print(W_est.sum())
    # print(acc)
    ###############################################################

    X2 = df.to_numpy()
    W_est = notears_linear(X2, lambda1=0., loss_type='l2', max_iter=5)
    print("W_est: {}".format(W_est))

    import pandas as pd
    W_out = pd.DataFrame(W_est)
    W_out.to_csv('entire_biobank_10_encode_age_max_2.csv')
    # X = read_file("X_mimic.txt", " ")
    # W_est = notears_linear(X, lambda1=0., loss_type='l2', max_iter=100, w_threshold=0.)
    print(W_est)

    ##############################
    # import pandas as pd
    # mimic3 = read_file("DIAGNOSES_ICD.csv")
    # mimic3 = pd.DataFrame(mimic3[1:], columns=["ROW_ID", "SUBJECT_ID", "HADM_ID", "SEQ_NUM", "ICD9_CODE"])
    # patients = mimic3['SUBJECT_ID'].to_list()
    # patients = list(set(patients))
    # print(len(patients))
    # icd = read_file("icd.csv")
    # icd = np.array(icd).astype(int)